# Stent Calculator 2021
Craig Bonsignore, July 2021

Derived from Open Stent design calculator at https://github.com/cbonsig/open-stent, originally developed in 2010.

In [1]:
# required libraries
import math
import sympy as sym

## Background

The functional properties of a self-expanding stent can be approximated using engineering properties of simple beams. This worksheet estimates strain and stiffness values for specific use conditions, given typical stent design input variables, such as strut width, strut length, wall thickness, and tubing dimensions. 

This worksheet is a [Jupyter Notebook](https://jupyter.org) which uses the Python symbolic math library [SymPy](https://www.sympy.org/) to derive formulas for strain, radial outward force, contact pressure, and equivalent hydrostatic pressure. The formulas are expressed here in mathematical notation, and also summarized in a format suitable for implementation in a spreadsheet. 

The spreadsheet version allows for easy experimentation and iteration. It can be accessed via Google Sheets: [Stent Calculator 2021](https://docs.google.com/spreadsheets/d/1WNo-heeC47Z9YqaJjs4vi74C_uU2YyMQA8cRFJYoxgw/edit#gid=0). Use File > Make a Copy to make changes, or File > Download to use in Excel.

## Design variables

This model assumes a stent design as shown below, typically laser cut from nitinol tubing. The design variables below describe the essential characteristics of such a stent, with uniform regularly repeating features. All formulas will be expressed in terms of these variables.

![design_diagram](images/stent.png)

In [2]:
D_o = sym.Symbol('D_o')   # tubing outer diameter
t   = sym.Symbol('t')     # tubing wall thickness
w   = sym.Symbol('w')     # strut width
a   = sym.Symbol('a')     # apex width
L_s = sym.Symbol('L_s')   # strut length, measured to inner apex
N   = sym.Symbol('N')     # number of uniformly spaced struts
D_x = sym.Symbol('D_x')   # expanded diameter of stent at outer diameter
D_v = sym.Symbol('D_v')   # vessel diameter
E_a = sym.Symbol('E_a')   # effective elastic modulus of material

## Derived variables

The variables below are used for intermediate calculations.

In [3]:
D_i     = D_o - 2*t         # tubing inner diameter
X       = D_x - D_o         # change in diameter, from tubing to full expansion
L       = L_s + a           # effective strut length
C_o     = sym.pi * D_o      # initial circumference, tube OD
C_x     = sym.pi * D_x      # expanded circumference, outer surface of stent
C_v     = sym.pi * D_v      # vessel diameter circumference
deltaX  = (C_x-C_o) / N     # change in circumference per strut, from tube to full expansion
deltaV  = (C_x-C_v) / N     # change in circumference per strut, from full exp to vessel
A_s     = L_s * w           # strut area
A_a     = a * w             # apex area, approximated
A_c     = A_s + 2*A_a       # approximate contact area per strut

## Section modulus

A strut laser cut from a tube has a moment of inertia defined by $I_y$ of the "sector of a hollow circle" (Roark's Formulas for Stress and Strain, 7th Edition, page 808). The section modulus $Z_y$ is derived from this value. The formula for $I_y$ is highlighted below.

![roarks](images/roarks_formulas.png)

In [4]:
R          = D_o / 2              # outer radius tube
two_alpha  = (w/C_o) * 2*sym.pi   # strut width relative to tubing circumference
alpha      = two_alpha / 2        # in radians
term1      = (R**3) * (t)
term2      = 1-((3*t)/(2*R))+((t**2)/(R**2))-((t**3)/(4*R**3))
term3      = alpha - (sym.sin(alpha) * sym.cos(alpha))
I_y        = term1 * term2 * term3
Z_y        = I_y / (w/2)

The relevant moment of inertia $I_y$ can now be expressed in terms of the design variables.

In [5]:
sym.factor(I_y)

-t*(-D_o + t)*(-D_o*sin(w/D_o)*cos(w/D_o) + w)*(D_o**2 - 2*D_o*t + 2*t**2)/(8*D_o)

## Beam deflection, expansion

$deltaX$, derived above, is the circumferential distance each strut moves when expanded by $X$ millimeters in diameter. This corresponds to the deflection of the beam, assuming the stent is expanded from the tubing diameter to the final diameter in a single step.

In [6]:
sym.factor(deltaX)

-pi*(D_o - D_x)/N

## Beam deflection, deployed

$deltaV$, also derived above, is the circumferential distance each strut moves when the stent is reduced in diameter ("oversized") from its fully expanded size to the smaller size of the vessel.

In [7]:
sym.factor(deltaV)

-pi*(D_v - D_x)/N

## Foreshortening at expanded diameter

$\theta_X$ is half the included angle formed by adjacent struts when the stent is fully expanded, assuming that struts are aligned with the longitudinal axis in the constrained state (at the diameter of the tubing).

In [8]:
theta_X = sym.atan( deltaX / L )
theta_X

atan((-pi*D_o + pi*D_x)/(N*(L_s + a)))

Foreshortening is the percentage change in length when the stent is expanded from its constrained diameter to its largest expanded diameter.

In [9]:
L_x      = L * sym.cos(theta_X)   # expanded strut length, axial component
FS_x     = (L-L_x) / L            # foreshortening
FS_x

(L_s + a - (L_s + a)/sqrt(1 + (-pi*D_o + pi*D_x)**2/(N**2*(L_s + a)**2)))/(L_s + a)

## Force when constrained at vessel diameter
A deflected strut resembles a beam fixed at one end, and free but guided at the other. Machinery's Handbook 26 (page 243), "case 17", provides formulas for load $W$ corresponding to a given deflection. In this scenario, deflection $deltaV$ is the change in circumference when the fully expanded stent is constrained in a vessel.
![machinery](images/machinerys_handbook.png)

In [10]:
W       = sym.Symbol('W')                # force as shown in Case 17
deflect = (W * L**3) / (12 * E_a * I_y)  # maximum deflection at free end
eq      = deflect - deltaV               # equation: deflection == deltaV
Fhoop   = sym.solve(eq,W)[0]             # solve for force, first solution
Fhoop   = sym.factor(Fhoop)
Fhoop

3*pi*E_a*t*(-D_o + t)*(D_v - D_x)*(-D_o*sin(2*w/D_o) + 2*w)*(D_o**2 - 2*D_o*t + 2*t**2)/(4*D_o*N*(L_s + a)**3)

This is the total magnitude of the force exerted by the strut when the diameter is constrained from the fully expanded diameter to the vessel diameter. This is a "hoop force", oriented in the cylindrical plane of the circumference of the stent. This force can be decomposed into a circumferentially oriented component ($\theta$, related to radial stiffness) and a longitudinally oriented component (related to axial stiffness).

In [11]:
Fhoop_theta = Fhoop * sym.cos(theta_X)
Fhoop_theta

3*pi*E_a*t*(-D_o + t)*(D_v - D_x)*(-D_o*sin(2*w/D_o) + 2*w)*(D_o**2 - 2*D_o*t + 2*t**2)/(4*D_o*N*sqrt(1 + (-pi*D_o + pi*D_x)**2/(N**2*(L_s + a)**2))*(L_s + a)**3)

Convert hoop force to true radial force, a multiple of $2\pi$.

In [12]:
F_trf      = 2 * sym.pi * Fhoop_theta
F_trf

3*pi**2*E_a*t*(-D_o + t)*(D_v - D_x)*(-D_o*sin(2*w/D_o) + 2*w)*(D_o**2 - 2*D_o*t + 2*t**2)/(2*D_o*N*sqrt(1 + (-pi*D_o + pi*D_x)**2/(N**2*(L_s + a)**2))*(L_s + a)**3)

## Contact pressure

The contact pressure is the outward force of the stent strut divided by outer surface area of the strut. This is the pressure "felt" by an endothelial cell in contact with the stent.

In [13]:
A_c            # contact area

L_s*w + 2*a*w

In [14]:
F_trf/A_c      # radial force / contact area

3*pi**2*E_a*t*(-D_o + t)*(D_v - D_x)*(-D_o*sin(2*w/D_o) + 2*w)*(D_o**2 - 2*D_o*t + 2*t**2)/(2*D_o*N*sqrt(1 + (-pi*D_o + pi*D_x)**2/(N**2*(L_s + a)**2))*(L_s + a)**3*(L_s*w + 2*a*w))

## Equivalent pressure

The equivalent pressure is the outward force of the stent divided by the surface of the area of the vessel in which the stent is placed. This is the equivalent hydrostatic pressure required to replace the effect of the stent.

In [15]:
A_v = C_v * L_x     # assuming length of vessel == length of expanded stent
A_v

pi*D_v*(L_s + a)/sqrt(1 + (-pi*D_o + pi*D_x)**2/(N**2*(L_s + a)**2))

In [16]:
F_trf/A_v

3*pi*E_a*t*(-D_o + t)*(D_v - D_x)*(-D_o*sin(2*w/D_o) + 2*w)*(D_o**2 - 2*D_o*t + 2*t**2)/(2*D_o*D_v*N*(L_s + a)**4)

## Strain when constrained in vessel

The formulation for stress is expressed in the Machinery's Handbook reference above. "Stress at support" is defined here as stress $\sigma$, and the effective elastic modulus is then used to estimate strain $\epsilon$.

In [17]:
sigma = (W * L) / (2 * Z_y)
epsilon = sigma / E_a
epsilon = sym.simplify(epsilon)
epsilon

-2*D_o*W*w*(L_s + a)/(E_a*t*(D_o*sin(2*w/D_o)/2 - w)*(D_o**3 - 3*D_o**2*t + 4*D_o*t**2 - 2*t**3))

The above formulation expresses strain as a function of the applied force $W$. The section above calculates the relationship between applied force $F$ and displacement $deltaV$. Now, substituting $F$ for the applied force $W$, strain can be expressed in terms of the design variables.

In [18]:
epsilon = sym.simplify(epsilon.subs(W,Fhoop_theta))
epsilon

-3*pi*w*(D_o - t)*(D_v - D_x)*(D_o**2 - 2*D_o*t + 2*t**2)/(N*sqrt((N**2*(L_s + a)**2 + pi**2*(D_o - D_x)**2)/(N**2*(L_s + a)**2))*(L_s + a)**2*(D_o**3 - 3*D_o**2*t + 4*D_o*t**2 - 2*t**3))

## Reformat formulas for spreadsheet implementation

The `sym.ccode()` method expresses symbolic formulas in C code. This format is conveniently similar to that required by Excel or Google Sheets to define spreadsheet formulas. Copy the formulas, replace `M_PI` with `PI()`, and replace each symbolic variable with the corresponding spreadsheet cell reference.

In [19]:
sym.ccode(FS_x) # foreshortening

'(L_s + a - (L_s + a)/sqrt(1 + pow(-M_PI*D_o + M_PI*D_x, 2)/(pow(N, 2)*pow(L_s + a, 2))))/(L_s + a)'

In [20]:
sym.ccode(epsilon) # strain

'-3*M_PI*w*(D_o - t)*(D_v - D_x)*(pow(D_o, 2) - 2*D_o*t + 2*pow(t, 2))/(N*sqrt((pow(N, 2)*pow(L_s + a, 2) + pow(M_PI, 2)*pow(D_o - D_x, 2))/(pow(N, 2)*pow(L_s + a, 2)))*pow(L_s + a, 2)*(pow(D_o, 3) - 3*pow(D_o, 2)*t + 4*D_o*pow(t, 2) - 2*pow(t, 3)))'

In [21]:
sym.ccode(F_trf) # absolute radial force

'(3.0/2.0)*pow(M_PI, 2)*E_a*t*(-D_o + t)*(D_v - D_x)*(-D_o*sin(2*w/D_o) + 2*w)*(pow(D_o, 2) - 2*D_o*t + 2*pow(t, 2))/(D_o*N*sqrt(1 + pow(-M_PI*D_o + M_PI*D_x, 2)/(pow(N, 2)*pow(L_s + a, 2)))*pow(L_s + a, 3))'

In [22]:
sym.ccode(L_x) # expanded length, to normalize radial force by length

'(L_s + a)/sqrt(1 + pow(-M_PI*D_o + M_PI*D_x, 2)/(pow(N, 2)*pow(L_s + a, 2)))'

In [23]:
sym.ccode(A_c) # contact area, to calculate contact pressure

'L_s*w + 2*a*w'

In [24]:
sym.ccode(A_v) # vessel area, to calculate hydrostatic equivalent pressure

'M_PI*D_v*(L_s + a)/sqrt(1 + pow(-M_PI*D_o + M_PI*D_x, 2)/(pow(N, 2)*pow(L_s + a, 2)))'

## Example design

A list $stent$ is created below, and populated with tuples containing example numerical values for each of the symbolically defined variables. The values in this list are then _substituted_ into the previously derived formulas using the `.subs()` method. 

In [25]:
stent = []                             # create an empty list
stent.append((D_o,1.915))              # tubing outer diameter, mm
stent.append((t, 0.170 - 0.059))       # wall thickness, mm (adjusted for polishing removal)
stent.append((w, 0.130 - 0.036))       # strut width, mm (adjusted for polishing removal)
stent.append((a, 0.130 - 0.036))       # apex width, mm (adjusted for polishing removal)
stent.append((L_s, 1.2))               # strut length, mm (inner apex distance)
stent.append((N, 42))                  # number of struts
stent.append((D_x,8.00))               # expanded diameter of stent, mm
stent.append((D_v,6.50))               # vessel diameter, mm
stent.append((E_a,40E3))               # elastic modulus, MPa
stent.append((sym.pi, math.pi))        # convert symbolic pi to numerical value

Sanity check on kerf width, the gap between struts in the constrained state, assuming all struts are parallel axially oriented on the tubing. (micron)

In [26]:
kerf_um = 1000 * ((C_o / N) - w)
kerf_um.subs(stent)

49.2416650386775

Verify the estimated strain when the expanded stent is constrained within the vessel. As strains approach and exceed 2% (0.02) this simple model diverges from reality.

In [27]:
strain = epsilon.subs(stent)
strain

0.0178255352127590

Estimated radial force, normalized by length. (N/mm)

In [28]:
RF = F_trf
RF_L = F_trf / L_x
RF_L.subs(stent)

0.777743195968729

Estimated contact pressure (N/mm^2 = MPa)

In [29]:
P_contact = RF.subs(stent) / A_c.subs(stent)
P_contact

7.27651381878234

Estimated equivalent hydrostatic pressure (mmHg)

In [30]:
P_hydro_mmHg = (RF.subs(stent) / A_v.subs(stent)) * 7500.62     # mmHg / MPa
P_hydro_mmHg

285.673630875932